Model1 training on 50000 noised labels: 0.4395 val_accuracy on 10 epochs

Model2 training on 50000 cleaned labels + vgg16 features extraction: 0.5717 val_acc on 10 epochs. 

## 0. Requirements

In [1]:
# Import required packages
!pip install opencv-python

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD, Adam
from keras import optimizers, regularizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D,Activation,GlobalAveragePooling2D,Input,Dropout,concatenate,BatchNormalization,Concatenate
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
from keras.layers import Layer
from __future__ import print_function
import keras
from keras.datasets import cifar10
import os
from tensorflow.keras.utils import to_categorical

You should consider upgrading via the 'c:\users\31557\anaconda3\python.exe -m pip install --upgrade pip' command.


## 1. Datasets

In [2]:
# [DO NOT MODIFY THIS CELL]

# load the images
n_img = 50000
n_noisy = 40000
n_clean_noisy = n_img - n_noisy
imgs = np.empty((n_img,32,32,3))
for i in range(n_img):
    img_fn = f'../data/images/{i+1:05d}.png'
    imgs[i,:,:,:]=cv2.cvtColor(cv2.imread(img_fn),cv2.COLOR_BGR2RGB)

# load the labels
clean_labels = np.genfromtxt('../data/clean_labels.csv', delimiter=',', dtype="int8")
noisy_labels = np.genfromtxt('../data/noisy_labels.csv', delimiter=',', dtype="int8")

target_vec = np.empty((n_img,10))
i = 0
for i in range(n_img):
    # The target vector consists of noisy labels
    if i < 10000:
        target_vec[i][int(clean_labels[i])-1] = 1
    else:
        target_vec[i][int(noisy_labels[i])-1] = 1

y_train_clean =  to_categorical(clean_labels, 10)
y_train_noisy =  to_categorical(noisy_labels[:10000],10)

x_train_clean = imgs[:10000].astype("float16")/255
x_train_full = imgs.astype("float16")/255

x_train_clean.astype("float16")
x_train_full.astype("float16")

y_train_full = target_vec

y_train_clean.shape, y_train_noisy.shape, x_train_clean.shape, x_train_full.shape,y_train_full.shape

((10000, 10), (10000, 10), (10000, 32, 32, 3), (50000, 32, 32, 3), (50000, 10))

## Model 1, base model

Trained on clean images and labels

In [3]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h1'


# _, (x_test, y_test) = cifar10.load_data()


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# RMSprop
opt = tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)


model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


if not data_augmentation:
    print('Not using data augmentation.')
#     model.fit(x_train_clean, y_train_clean,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_data=(X_test, y_test),
#               shuffle=True)

else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
    datagen.fit(x_train_clean)

    
    model.fit(datagen.flow(x_train_clean, y_train_clean, batch_size=32,subset='training'),
         validation_data=datagen.flow(x_train_clean, y_train_clean,batch_size=8, subset='validation'),
          epochs=20)

C:\Users\31557\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Using real-time data augmentation.
Epoch 1/20
250/250 [==============================] - 12s 44ms/step - loss: 2.1612 - accuracy: 0.1945 - val_loss: 2.0092 - val_accuracy: 0.2675
Epoch 2/20
250/250 [==============================] - 11s 44ms/step - loss: 1.9732 - accuracy: 0.2707 - val_loss: 1.8857 - val_accuracy: 0.3230
Epoch 3/20
250/250 [==============================] - 11s 45ms/step - loss: 1.8824 - accuracy: 0.3040 - val_loss: 1.8050 - val_accuracy: 0.3320
Epoch 4/20
250/250 [==============================] - 11s 45ms/step - loss: 1.8181 - accuracy: 0.3228 - val_loss: 1.7469 - val_accuracy: 0.3625
Epoch 5/20
250/250 [==============================] - 11s 45ms/step - loss: 1.7691 - accuracy: 0.3458 - val_loss: 1.6842 - val_accuracy: 0.3850
Epoch 6/20
250/250 [==============================] - 11s 43ms/step - loss: 1.7350 - accuracy: 0.3599 - val_loss: 1.6450 - val_accuracy: 0.4085
Epoch 7/20
250/250 [==============================] - 10s 42ms/step - loss: 1.6886 - accuracy: 0.3784

Trained on noisy labels

In [4]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h1_noisy'




model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# RMSprop
opt = tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)


model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


if not data_augmentation:
    print('Not using data augmentation.')
#     model.fit(x_train_clean, y_train_clean,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_data=(X_test, y_test),
#               shuffle=True)

else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
    datagen.fit(x_train_full)

    
    model.fit(datagen.flow(x_train_full, y_train_full, batch_size=32,subset='training'),
         validation_data=datagen.flow(x_train_full, y_train_full,batch_size=8, subset='validation'),
          epochs=10)

Using real-time data augmentation.
Epoch 1/10
1250/1250 [==============================] - 56s 44ms/step - loss: 2.2857 - accuracy: 0.1362 - val_loss: 2.1705 - val_accuracy: 0.2893
Epoch 2/10
1250/1250 [==============================] - 55s 44ms/step - loss: 2.2681 - accuracy: 0.1575 - val_loss: 2.1037 - val_accuracy: 0.3265
Epoch 3/10
1250/1250 [==============================] - 55s 44ms/step - loss: 2.2574 - accuracy: 0.1696 - val_loss: 2.0969 - val_accuracy: 0.3470
Epoch 4/10
1250/1250 [==============================] - 56s 44ms/step - loss: 2.2510 - accuracy: 0.1789 - val_loss: 2.0330 - val_accuracy: 0.3728
Epoch 5/10
1250/1250 [==============================] - 56s 44ms/step - loss: 2.2459 - accuracy: 0.1834 - val_loss: 1.9933 - val_accuracy: 0.3901
Epoch 6/10
1250/1250 [==============================] - 56s 45ms/step - loss: 2.2416 - accuracy: 0.1871 - val_loss: 1.9496 - val_accuracy: 0.4061
Epoch 7/10
1250/1250 [==============================] - 56s 45ms/step - loss: 2.2395 - ac

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 64)       

## 3. Model 2, cleaning label + classifier


In [6]:
# cleaning labels

In [7]:

# base_cnn = model.layers[-5:]

# for layer in base_cnn:
#     layer.trainable=False

from keras.applications.vgg16 import VGG16
base_model_vgg16=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(32,32,3)),pooling="max" ) 
base_model_vgg16.trainable = False

from keras.applications.vgg19 import VGG19
base_model_vgg19=VGG19(weights='imagenet',include_top=False,input_tensor=Input(shape=(32,32,3)),pooling="max" ) 
base_model_vgg19.trainable = False

class LastLayer(Layer):

    def __init__(self, **kwargs):
        self.output_dim = 10
        super(LastLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        super(LastLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, gxy, y):
        return K.maximum(K.minimum(gxy + y, 1),0)


def label_loss(true_label, cleaned_label):
    return K.sum(K.abs(cleaned_label - true_label))



def model_2(base_cnn):
    
    img_input = Input(shape=(32, 32, 3))
    noisy_label = Input(shape = (10))
    
    img_vec = base_cnn(img_input)
    
    noisy_l = Dense(10)(noisy_label)
    img_vec = Dense(256)(img_vec)

    # combine
    x = Concatenate(axis=-1)([noisy_l, img_vec])
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    out = Dense(10, activation = 'softmax')(x)
    # out = LastLayer()(out, noisy_label)
    
    model = Model([img_input, noisy_label], out)

    return model


model2 = model_2(base_model_vgg16)

Adam = tf.keras.optimizers.Adam(0.001)

model2.compile(loss=label_loss, metrics=['acc'], optimizer=Adam)

model2.fit([x_train_clean, y_train_noisy], y_train_clean , batch_size = 128, epochs = 50)


Epoch 1/50
79/79 [==============================] - 15s 176ms/step - loss: 199.3929 - acc: 0.2576
Epoch 2/50
79/79 [==============================] - 13s 163ms/step - loss: 159.1855 - acc: 0.4210
Epoch 3/50
79/79 [==============================] - 13s 163ms/step - loss: 139.7927 - acc: 0.4955
Epoch 4/50
79/79 [==============================] - 13s 164ms/step - loss: 130.9886 - acc: 0.5193
Epoch 5/50
79/79 [==============================] - 13s 163ms/step - loss: 121.4623 - acc: 0.5514
Epoch 6/50
79/79 [==============================] - 13s 163ms/step - loss: 115.6424 - acc: 0.5746
Epoch 7/50
79/79 [==============================] - 13s 163ms/step - loss: 114.3628 - acc: 0.5739
Epoch 8/50
79/79 [==============================] - 13s 163ms/step - loss: 110.2011 - acc: 0.5871
Epoch 9/50
79/79 [==============================] - 13s 163ms/step - loss: 105.6384 - acc: 0.6029
Epoch 10/50
79/79 [==============================] - 13s 163ms/step - loss: 103.2040 - acc: 0.6111
Epoch 11/50
79/79 [

In [8]:
cleaned_label = model2.predict([x_train_full,y_train_full])

In [9]:
cleaned_label.shape

(50000, 10)

In [10]:
cleaned_labels = np.argmax(cleaned_label,axis=1 )
cleaned_labels = to_categorical(cleaned_labels,10)

In [11]:
cleaned_labels[0], sum(cleaned_labels[0])

(array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32), 1.0)

In [12]:


def model_3(base_cnn):
    
    img_input = Input(shape=(32, 32, 3))
    
    img_vec = base_cnn(img_input)
    
    x = Dense(256)(img_vec)

    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    out = Dense(10, activation = 'softmax')(x)
    # out = LastLayer()(out, noisy_label)
    
    model = Model(img_input, out)

    return model

model3 = model_3(base_model_vgg16)

loss = tf.keras.losses

Adam = tf.keras.optimizers.Adam(0.001)

model3.compile(loss="categorical_crossentropy", metrics=['acc'], optimizer=Adam)

datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
datagen.fit(x_train_full)

model3.fit(datagen.flow(x_train_full, cleaned_labels, batch_size=32,subset='training'),
           validation_data=datagen.flow(x_train_full, cleaned_labels,batch_size=8, subset='validation'),
           batch_size = 128, 
           epochs = 10)





Epoch 1/10
1250/1250 [==============================] - 95s 75ms/step - loss: 1.7597 - acc: 0.3938 - val_loss: 1.3234 - val_acc: 0.5429
Epoch 2/10
1250/1250 [==============================] - 93s 74ms/step - loss: 1.5055 - acc: 0.4800 - val_loss: 1.2936 - val_acc: 0.5474
Epoch 3/10
1250/1250 [==============================] - 94s 75ms/step - loss: 1.4644 - acc: 0.4961 - val_loss: 1.2607 - val_acc: 0.5648
Epoch 4/10
1250/1250 [==============================] - 95s 76ms/step - loss: 1.4448 - acc: 0.5064 - val_loss: 1.2510 - val_acc: 0.5684
Epoch 5/10
1250/1250 [==============================] - 93s 74ms/step - loss: 1.4418 - acc: 0.5047 - val_loss: 1.2619 - val_acc: 0.5600
Epoch 6/10
1250/1250 [==============================] - 94s 75ms/step - loss: 1.4272 - acc: 0.5104 - val_loss: 1.2698 - val_acc: 0.5687
Epoch 7/10
1250/1250 [==============================] - 96s 77ms/step - loss: 1.4223 - acc: 0.5127 - val_loss: 1.2463 - val_acc: 0.5690
Epoch 8/10
1250/1250 [==========================

## Add tensorboard for visualization

In [ ]:
%load_ext tensorboard
import datetime
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# model = label_nn()
tenserboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
# history1 = model.fit([clean_imgs, noisy], clean, batch_size = 128, epochs =5, callbacks = [tb])

In [16]:
def model_4(base_cnn):
    
    img_input = Input(shape=(32, 32, 3))
    
    img_vec = base_cnn(img_input)
    

    x = Dense(256, activation = 'relu')(img_vec)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    out = Dense(10, activation = 'softmax')(x)
    # out = LastLayer()(out, noisy_label)
    
    model = Model(img_input, out)

    return model

model4 = model_4(base_model_vgg16)

# loss = tf.keras.losses

Adam = tf.keras.optimizers.Adam(0.001)

model4.compile(loss="categorical_crossentropy", metrics=['acc'], optimizer=Adam)

datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
datagen.fit(x_train_full)

file_path = "../output/model4_history1"
checkpoint = ModelCheckpoint(file_path, monitor='acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')

callback = [checkpoint, tenserboard]

model4.fit(datagen.flow(x_train_full, cleaned_labels, batch_size=32,subset='training'),
           validation_data=datagen.flow(x_train_full, cleaned_labels,batch_size=8, subset='validation'),
           batch_size = 128,
           callbacks=callback
           epochs = 10)




Epoch 1/10
1250/1250 [==============================] - 90s 71ms/step - loss: 1.7784 - acc: 0.3805 - val_loss: 1.3355 - val_acc: 0.5464
Epoch 2/10
1250/1250 [==============================] - 91s 73ms/step - loss: 1.5408 - acc: 0.4642 - val_loss: 1.2870 - val_acc: 0.5550
Epoch 3/10
1250/1250 [==============================] - 90s 72ms/step - loss: 1.4921 - acc: 0.4805 - val_loss: 1.2787 - val_acc: 0.5583
Epoch 4/10
1250/1250 [==============================] - 94s 75ms/step - loss: 1.4679 - acc: 0.4886 - val_loss: 1.2504 - val_acc: 0.5650
Epoch 5/10
1250/1250 [==============================] - 97s 78ms/step - loss: 1.4614 - acc: 0.4957 - val_loss: 1.2563 - val_acc: 0.5683
Epoch 6/10
1250/1250 [==============================] - 93s 75ms/step - loss: 1.4462 - acc: 0.5005 - val_loss: 1.2415 - val_acc: 0.5715
Epoch 7/10
 628/1250 [==============>...............] - ETA: 35s - loss: 1.4345 - acc: 0.5030

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir logs/fit